# Kùzu Graph Store

In [1]:
# My OpenAI Key
import os

os.environ["OPENAI_API_KEY"] = "INSERT OPENAI KEY"

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare for Kùzu

In [3]:
# Clean up all the directories used in this notebook
import shutil
shutil.rmtree("test1", ignore_errors=True)
shutil.rmtree("test2", ignore_errors=True)
shutil.rmtree("test3", ignore_errors=True)

In [4]:
%pip install kuzu
import kuzu
db = kuzu.Database("test1")


[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Using Knowledge Graph with KuzuGraphStore

In [5]:
from llama_index.graph_stores import KuzuGraphStore
graph_store = KuzuGraphStore(db)

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


#### Building the Knowledge Graph

In [6]:
from llama_index import (
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    KnowledgeGraphIndex,
)

from llama_index.llms import OpenAI
from IPython.display import Markdown, display
import kuzu

In [7]:
documents = SimpleDirectoryReader(
    "../../../../examples/paul_graham_essay/data"
).load_data()

In [8]:
# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors

llm = OpenAI(temperature=0, model="text-davinci-002")
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

In [9]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    storage_context=storage_context,
    service_context=service_context,
)

#### [Optional] Try building the graph and manually add triplets!

#### Querying the Knowledge Graph

In [10]:
query_engine = index.as_query_engine(include_text=False, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about Interleaf
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf', 'history', 'company', 'software']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made software for', 'creating documents']
Interleaf ['added', 'scripting language']
Interleaf ['teaches', 'what not to do']
software ['is', 'WYSIWYG site builder']


In [11]:
display(Markdown(f"<b>{response}</b>"))

<b>
Interleaf was a software company that made software for creating documents. They added a scripting language to their software to make it more user-friendly. Interleaf also taught users what not to do when using their software.</b>

In [12]:
query_engine = index.as_query_engine(include_text=True, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about what the author worked on at Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about what the author worked on at Interleaf
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf', 'author', 'work']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 5b979e1e-eeaf-4682-b56a-cda4211949e8: each student had. But the Accademia wasn't teaching me anything except Italia...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: a16f9313-ebe5-4190-94b8-d9987240520f: learned some useful things at Interleaf, though they were mostly about what n...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made software for', 'creating documents

In [13]:
display(Markdown(f"<b>{response}</b>"))

<b>
The author worked on software that created documents, and Interleaf had added a scripting language. The author also learned that it is better for technology companies to be run by product people than sales people, and that there is not much overlap between conventional office hours and the optimal time for hacking.</b>

#### Query with embeddings

In [14]:
# NOTE: can take a while!
db = kuzu.Database("test2")
graph_store = KuzuGraphStore(db)
storage_context = StorageContext.from_defaults(graph_store=graph_store)
new_index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=2,
    service_context=service_context,
    storage_context=storage_context,
    include_embeddings=True,
)

In [15]:
rel_map  = graph_store.get_rel_map()

In [16]:
# query using top 3 triplets plus keywords (duplicate triplets are removed)
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)
response = query_engine.query(
    "Tell me more about what the author worked on at Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about what the author worked on at Interleaf
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf', 'author', 'work']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 5b979e1e-eeaf-4682-b56a-cda4211949e8: each student had. But the Accademia wasn't teaching me anything except Italia...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: a16f9313-ebe5-4190-94b8-d9987240520f: learned some useful things at Interleaf, though they were mostly about what n...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made software for', 'creating documents

In [17]:
display(Markdown(f"<b>{response}</b>"))

<b>
The author worked on software that created documents, and Interleaf had added a scripting language. The author also learned that it is better for technology companies to be run by product people than sales people, and that there is not much overlap between conventional office hours and the optimal time for hacking.</b>

#### Visualizing the Graph

In [18]:
## create graph
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("kuzugraph_draw.html")

kuzugraph_draw.html


#### [Optional] Try building the graph and manually add triplets!

In [19]:
from llama_index.node_parser import SimpleNodeParser

In [20]:
node_parser = SimpleNodeParser()

In [21]:
nodes = node_parser.get_nodes_from_documents(documents)

In [22]:
# initialize an empty database
db = kuzu.Database("test3")
graph_store = KuzuGraphStore(db)
storage_context = StorageContext.from_defaults(graph_store=graph_store)
index = KnowledgeGraphIndex(
    [],
    service_context=service_context,
    storage_context=storage_context,
)

In [23]:
# add keyword mappings and nodes manually
# add triplets (subject, relationship, object)

# for node 0
node_0_tups = [
    ("author", "worked on", "writing"),
    ("author", "worked on", "programming"),
]
for tup in node_0_tups:
    index.upsert_triplet_and_node(tup, nodes[0])

# for node 1
node_1_tups = [
    ("Interleaf", "made software for", "creating documents"),
    ("Interleaf", "added", "scripting language"),
    ("software", "generate", "web sites"),
]
for tup in node_1_tups:
    index.upsert_triplet_and_node(tup, nodes[1])

In [24]:
query_engine = index.as_query_engine(include_text=False, response_mode="tree_summarize")
response = query_engine.query(
    "Tell me more about Interleaf",
)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: Tell me more about Interleaf
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Interleaf', 'history', 'company', 'software']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted relationships: The following are knowledge triplets in max depth 2 in the form of `subject [predicate, object, predicate_next_hop, object_next_hop ...]`
Interleaf ['made software for', 'creating documents']
Interleaf ['added', 'scripting language']
software ['generate', 'web sites']


In [25]:
str(response)

'\nInterleaf was a software company that made software for creating documents. They added a scripting language to their software to generate web sites.'